In [1]:
import requests
import json
import time
import pandas as pd
from pandas import DataFrame

#FYI, If you want to use google maps you have to create an account to get the API
#We create the class, inofrmation as endpointurl or apikey is given by google, 
class GooglePlaces(object):
    def __init__(self, apiKey):
        super(GooglePlaces, self).__init__()
        self.apiKey = apiKey
 
    def search_places_by_coordinate(self, location, radius, types):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        places = []
        params = {
            'location': location,
            'radius': radius,
            'types': ["restaurant","bar"],
            'key': self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        places.extend(results['results'])
        time.sleep(5)
        while "next_page_token" in results:
            params['pagetoken'] = results['next_page_token'],
            res = requests.get(endpoint_url, params=params)
            results = json.loads(res.content)
            places.extend(results['results'])
            time.sleep(5)
        return places
 
    def get_place_details(self, place_id, fields):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json"
        params = {
            'placeid': place_id,
            'fields': ",".join(fields),
            'key': self.apiKey,

        }
        res = requests.get(endpoint_url, params=params)
        place_details =  json.loads(res.content)
        return place_details
    
    
#I delete my API key - Andrea Palomino. The coordinates that I use are the center of the zone that I wanted the results

api = GooglePlaces("Your API key")

places = api.search_places_by_coordinate("-12.069338, -76.978985","800","restaurant")    
    
fields = ['name','address_component', 'formatted_address', 'rating', 'review','geometry']

#Here we call the previous functions. From Google maps we can also get another 
#info as the review's author or even pics of the places. 

locales_ratings_locacion = []
for place in places:
    details = api.get_place_details(place['place_id'], fields)
    print(details)
    try:
        name = details['result']['name']
    except KeyError:
        name = ""
    try:
        reviews = details['result']['reviews']
    except KeyError:
        reviews = []
    try:
        geometry = details['result']['geometry']
    except KeyError:
        geometry = []
    ratings = []
    usuarios=[]
    for review in reviews:
        author_name = review['author_name']
        usuarios.append(author_name)
        rating = review['rating']
        ratings.append(rating)
       
    locales_ratings_locacion.append((name,usuarios,ratings,geometry))
    
locales_ratings_locacion